# Filtragem dos dados
Os dados originais para a entidade de relacionamento UserAnime é bastante grande, sendo impossível carregá-la inteiramente na memória. Portanto, aqui aplicamos métodos de filtragem para reduzir o banco de dados original.

In [1]:
import pandas as pd

In [3]:
users = pd.read_csv('./original_data/users_cleaned.csv');
animes = pd.read_csv('./original_data/anime_cleaned.csv');

## Método 1: Filtro proporcional
Filter apenas as relações user_anime que possuam uma ampla proporção de episódios assistidos pelo total de episódios lançados de um anime.

In [3]:
def anime_is_eligible(user_anime, watched_proportion):
    anime = animes[animes.anime_id == user_anime.anime_id].iloc[[0]]
    if (anime.episodes.item() > 0):
        return user_anime.my_watched_episodes/anime.episodes.item() >= watched_proportion
    else: return True

In [1]:
# Initializing user_anime Data Frame
users_animes_prop = pd.DataFrame();

# The size of each file chunk (if this is altered, we can't print a progress bar, so be careful)
chunksize = 10 ** 6

# If filtered, the minimum proportion of watched episodes per total number of episodes
proportion = 0.8

count = 0
print("Filtering UserAnime List proportionally, hold on...")
for chunk in pd.read_csv('./original_data/animelists_cleaned.csv', chunksize=chunksize):
    for index, row in chunk.iterrows():
        if anime_is_eligible(row, proportion):
            users_animes_prop = users_animes_prop.append(row)
            current_size = len(users_animes.index)
            print("Current Data Frame qtd of rows: %d" % current_size, end="\r")
            
    count += 1
    progress = (count/32) * 100
    print()
    print("%.2f%% read..." % progress, end="\r")
print("Done!")

## Método 2: Subconjunto aleatório
Escolhe linhas aleatórias da tabela de relacionamento UserAnime. Este método é preferível ao primeiro, uma vez que é mais eficiente e representa melhor os dados.

In [4]:
# Initializing user_anime Data Frame
users_animes_rand = pd.DataFrame();

# The size of each file chunk (if this is altered, we can't print a progress bar, so be careful)
chunksize = 10 ** 6

# Number of random samples per chunk
random_size = 100000

count = 0
print("Filtering UserAnime List randomily, hold on...")
for chunk in pd.read_csv('./original_data/animelists_cleaned.csv', chunksize=chunksize):
    users_animes_rand = users_animes_rand.append(chunk.sample(n=random_size, replace=True))
            
    count += 1
    progress = (count/32) * 100
    size = len(users_animes_rand.index)
    print("Current Data Frame row size: %d | %.2f%% completed..." % (size, progress), end="\r")
print()
print("Done! Run next cell to save Data Frame into a CSV file.")

Filtering UserAnime List randomily, hold on...
Current Data Frame row size: 3200000 | 100.00% completed...
Done! Run next cell to save Data Frame into a CSV file.


In [5]:
# Save random filtered to CSV file
users_animes_rand.to_csv("./manipulated_data/user_anime.csv")